In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from scipy.stats import chi2_contingency
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer  # For imputation
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE

In [2]:
# 1. Load data
feature_names = [
    "h3k_chromosome", "h3k_start", "h3k_end", "mvalue_h3k", "avalue_h3k", "pvalue_h3k", "peak_group_h3k",
    "normalized_density_1_h3k", "normalized_density_2_h3k", "chr_tf", "start_tf", "end_tf", "mvalue_tf",
    "avalue_tf", "pvalue_tf", "peak_group_tf", "normalized_density_1_tf", "normalized_density_2_tf",
    "annotation_gene_tf", "distance_to_TSS_tf", "symbol_tf", "gene_name_tf", "overlap"
]
df = pd.read_csv(
    '/trinity/home/atheodosiadou/macro/raw_data/ChIP_comb/ChIP_comb_merged_per_condition/bed_intersect/H3K27ac_EGR2_IL4/H3K27ac_EGR2_IL4_ctrl_overlap_modified.bed',
    sep="\t", header=None
)
df.columns = feature_names

# Function to extract chromosome number
def extract_chromosome_number(chromosome_string):
    """Extracts the chromosome number from a string like 'chr1', 'chrX', etc."""
    try:
        # Try to extract the number directly
        return int(chromosome_string[3:])  # Remove "chr" and convert to integer
    except ValueError:
        # Handle cases like 'chrX', 'chrY', etc.
        if chromosome_string[3:] == 'X':
            return 23
        elif chromosome_string[3:] == 'Y':
            return 24
        else:
            return np.nan  # Or some other appropriate value for unknown chromosomes

# Apply the function to the chromosome columns
df["h3k_chromosome"] = df["h3k_chromosome"].apply(extract_chromosome_number)
df["chr_tf"] = df["chr_tf"].apply(extract_chromosome_number)


# h3k_mapping = {
#     'H3K27ac_IL4.macs3.broad_peaks_filtered_unique': 0, 
#     'merged_common': 1,
#     'H3K27ac_ctrl.macs3.broad_peaks_filtered_unique': 2
# }

# tf_mapping = {
#     'EGR2_IL4.macs3_peaks_filtered_unique': 0,
#     'EGR2_ctrl.macs3_peaks_filtered_unique': 1,
#     'merged_common': 2,
#     '.': -1  # for missing values
# }

# def annotation_encoder(val):
#     if pd.isna(val) or val == ".":
#         return -1
#     val_lower = str(val).lower()
#     if "promoter_(<=1kb)" in val_lower:
#         return 0
#     elif "promoter_(1-2kb)" in val_lower:
#         return 1
#     elif "promoter_(2-3kb)" in val_lower:
#         return 2
#     elif "intron" in val_lower:
#         return 3
#     elif "exon" in val_lower:
#         return 4
#     elif "5'utr" in val_lower:
#         return 5
#     elif "3'utr" in val_lower:
#         return 6
#     elif "distal_intergenic" in val_lower:
#         return 7
#     elif "downstream" in val_lower:
#         return 8
#     else:
#         return -1


# # Apply the mappings
# df['peak_group_h3k'] = df['peak_group_h3k'].map(h3k_mapping)
# df['peak_group_tf'] = df['peak_group_tf'].map(tf_mapping)
# df['annotation_gene_tf'] = df['annotation_gene_tf'].apply(annotation_encoder)
df['overlap'] = df['overlap'].apply(lambda x: 1 if int(x) > 0 else 0)

In [3]:
# 2. Binarize overlap and mvalue_h3k
df['overlap'] = (df['overlap'].astype(int) == 1)
df['mvalue_h3k'] = (df['mvalue_h3k'].astype(int) == 1)

# # 3. One-hot encode categorical columns (add more if needed)
categorical_cols = ['peak_group_h3k', 'peak_group_tf', 'annotation_gene_tf']
df_encoded = pd.get_dummies(df[categorical_cols], prefix=categorical_cols)
df_apriori = pd.concat([df[['mvalue_h3k', 'overlap']], df_encoded], axis=1)
df_apriori = df_apriori.astype(bool)

# 4. Run apriori
frequent_itemsets = apriori(df_apriori, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

# 5. (Optional) Direct association: contingency table and chi-square test
ct = pd.crosstab(df['mvalue_h3k'], df['overlap'])
print("\nContingency table:\n", ct)
chi2, p, dof, expected = chi2_contingency(ct)
print(f"Chi2: {chi2}, p-value: {p}")

                                           antecedents  \
0                                         (mvalue_h3k)   
1                                         (mvalue_h3k)   
2                                         (mvalue_h3k)   
3                                         (mvalue_h3k)   
4                                         (mvalue_h3k)   
..                                                 ...   
210     (overlap, annotation_gene_tf_Promoter_(<=1kb))   
211  (peak_group_tf_merged_common, peak_group_h3k_m...   
212             (peak_group_tf_merged_common, overlap)   
213              (annotation_gene_tf_Promoter_(<=1kb))   
214                      (peak_group_tf_merged_common)   

                                           consequents   support  confidence  \
0                                            (overlap)  0.021854    0.566038   
1    (peak_group_h3k_H3K27ac_IL4.macs3.broad_peaks_...  0.023267    0.602630   
2                       (peak_group_h3k_merged_common)  0.01534

In [4]:
# Rows where both mvalue_h3k and overlap are True
strongly_associated_rows = df[(df['mvalue_h3k']) & (df['overlap'])]

print(strongly_associated_rows)

       h3k_chromosome  h3k_start   h3k_end  mvalue_h3k  avalue_h3k  \
67                  1    2289963   2291903        True     7.01415   
68                  1    2298847   2301671        True     6.53395   
69                  1    2314098   2315633        True     5.63661   
104                 1    6357336   6360755        True     7.36311   
105                 1    6357336   6360755        True     7.36311   
...               ...        ...       ...         ...         ...   
44896              22   38201422  38216480        True     6.57555   
45170              22   45328535  45329933        True     5.72561   
45256              22   49950692  49954887        True     5.32012   
45257              22   49950692  49954887        True     5.32012   
45285              22   50417161  50418609        True     6.16867   

         pvalue_h3k                                 peak_group_h3k  \
67     2.898982e-15                                  merged_common   
68     1.110592e-09